In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# UET


In [ ]:
programs = [
        "chuong-trinh-dao-tao-nganh-ky-thuat-dieu-khien-va-tu-dong-hoa-7",
        "chuong-trinh-dao-tao-nganh-cong-nghe-nong-nghiep-7",
        "chuong-trinh-dao-tao-nganh-thong-thong-tin-21",
        "chuong-trinh-dao-tao-nganh-mang-may-tinh-va-truyen-thong-du-lieu-13",
        "chuong-trinh-dao-tao-nganh-cong-nghe-thong-tin-10",
        "chuong-trinh-dao-tao-nganh-cong-nghe-thong-tin-dinh-huong-thi-truong-nhat-ban-10",
        "chuong-trinh-dao-tao-nganh-tri-tue-nhan-tao-6",
        "chuong-trinh-dao-tao-nganh-ky-thuat-may-tinh-10",
        "chuong-trinh-dao-tao-nganh-co-ky-thuat-24",
        "chuong-trinh-dao-tao-nganh-ky-thuat-nang-luong-10",
        "chuong-trinh-dao-tao-nganh-cong-nghe-ky-thuat-xay-dung-10",
        "chuong-trinh-dao-tao-nganh-cong-nghe-hang-khong-vu-tru-9",
        "chuong-trinh-dao-tao-nganh-thiet-ke-cong-nghiep-va-hoa-3",
        "chuong-trinh-dao-tao-cong-nghe-vat-lieu-va-vi-dien-tu-nganh-cong-nghe-vat-lieu-3",
        "chuong-trinh-dao-tao-cong-nghe-ky-thuat-sinh-hoc-nganh-cong-nghe-sinh-hoc-3",
        "chuong-trinh-dao-tao-nganh-ky-thuat-robot-7",
        "chuong-trinh-dao-tao-nganh-cong-nghe-ky-thuat-dien-tu-vien-thong-3",
        "chuong-trinh-dao-tao-nganh-vat-ly-ky-thuat-10",
        "chuong-trinh-dao-tao-nganh-cong-nghe-hang-khong-vu-tru-9",
]

In [36]:
for program in programs:
    url = f"https://uet.vnu.edu.vn/{program}"
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')

        # Chuyển bảng HTML thành DataFrame
        df = pd.read_html(str(table), header=0)[0]
        
    # Data Processing
    df = df.fillna(0)    
    if len(df.columns.tolist()) > 8:
        temp = df.iloc[:, 8]
        df = pd.concat([df.iloc[:, :7], temp], axis=1)
    new_columns = ['STT', 'Mã học phần', 'Học phần (ghi bằng tiếng Việt và tiếng Anh)', 'Số tín chỉ', 'Số giờ tín chỉ', 'Số giờ tín chỉ.1', 'Số giờ tín chỉ.2', 'Mã số học phần tiên quyết']
    df.columns = new_columns
    
    def clean_stt(stt):
        stt = str(stt)
        # Xóa dấu chấm cuối chuỗi
        if stt.endswith('.'):
            return stt[:-1]
        # Xử lý trường hợp đặc biệt khác nếu có
        return stt
    df['STT'] = df['STT'].apply(clean_stt)    

    result = []
    index_1 = None
    index_2 = None
    index_3 = None
    index_4 = None
    check = 0

    for _, row in df.iterrows():
        if row['STT'] == 0:
            continue
        if "Khối kiến thức" in str(row['Mã học phần']) and "." not in row['STT']: 
            # Phát hiện mục khối kiến thức tổng (cấp 1)
            check = 1
            index_1 = {
                "id": str(row['STT']),
                "general_knowledge": str(row['Học phần (ghi bằng tiếng Việt và tiếng Anh)']),
                "number_of_credits": str(row['Số tín chỉ']),
                "content": []
            }
            result.append(index_1)  
            continue
        elif "." in row['STT']:
            # Các nội dung block con cấp 2, 3,...
            level_depth = str(row['STT']).count(".")
            
            if level_depth == 1:
                # Block cấp 2
                check = 2
                index_2 = {
                    "id": str(row['STT']),
                    "general_knowledge": str(row['Học phần (ghi bằng tiếng Việt và tiếng Anh)']),
                    "number_of_credits": str(row['Số tín chỉ']),
                    "content": []
                }
                if index_2:
                    index_1['content'].append(index_2)
            elif level_depth == 2:
                # Block cấp 3
                check = 3
                index_3 = {
                    "id": str(row['STT']),
                    "general_knowledge": str(row['Học phần (ghi bằng tiếng Việt và tiếng Anh)']),
                    "content": []
                }
                if index_3:
                    index_2['content'].append(index_3)
            else:
                # Block cấp 4
                check = 4
                index_4 = {
                    "id": str(row['STT']),
                    "general_knowledge": str(row['Học phần (ghi bằng tiếng Việt và tiếng Anh)']),
                    "number_of_credits": str(row['Số tín chỉ']),
                    "content": []
                }
                if index_4:
                    index_3['content'].append(index_4)
        elif row['STT'].isdigit():
            course = {
                "id": str(row['STT']),
                "course_code": str(row['Mã học phần']),
                "course": str(row['Học phần (ghi bằng tiếng Việt và tiếng Anh)']),
                "number_of_credits": str(row['Số tín chỉ']),
                "credits_for_theory": str(row['Số giờ tín chỉ']),
                "credits_for_practice": str(row['Số giờ tín chỉ.1']),
                "credits_for_selflearn": str(row['Số giờ tín chỉ.2']),
                "prerequisite_course_code": str(row['Mã số học phần tiên quyết'])
            }
            if index_4 and check == 4:
                index_4["content"].append(course)
            elif index_3 and check == 3:
                index_3["content"].append(course)
            elif index_2 and check == 2:
                index_2["content"].append(course)
            elif index_1 and check == 1:
                index_1["content"].append(course)

            
    with open(f'uet/{program}.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

d:\longb_d\DABL\.conda\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
d:\longb_d\DABL\.conda\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'uet.vnu.edu.vn'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\longb\AppData\Local\Temp\ipykernel_15716\901070530.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table), header=0)[0]
